In [0]:
%sql
select * from bronze.crm_cust_info 

In [0]:
---CHECK FOR NULLS OR DUPLICATE IN PRIMARY KEY
---EXPECTED NULL

In [0]:
%sql
select cst_id,count(*) from bronze.crm_cust_info group by cst_id having count(*) > 1 or cst_id is null

In [0]:
---query to data transformation and cleansing
---remove duplicate from primary key
---remove trim spaces
---data standardization & consistency

In [0]:
%sql
select * from (
select *,row_number() over(partition by cst_id order by cst_create_date desc) as flag_last from bronze.crm_cust_info 
)
where flag_last = 1 

#### SILVER LAYER TABLE INGESTION AND QC

### crm_cust_info

In [0]:
%sql
insert overwrite silver.crm_cust_info (
    cst_id, cst_key, cst_firstname, cst_lastname, cst_marital_status, cst_gndr, cst_create_date
  )
  select
    cst_id,
    cst_key,
    trim(cst_firstname) as cst_firstname,
    trim(cst_lastname) as cst_lastname,
    CASE
      WHEN UPPER(cst_marital_status) = 'S' then 'Single'
      when UPPER(cst_marital_status) = 'M' then 'Married'
      ELSE 'n/a'
    END as cst_marital_status,---normalize marital status values to readable format
    CASE
      WHEN UPPER(cst_gndr) = 'F' then 'Female'
      when UPPER(cst_gndr) = 'M' then 'Male'
      ELSE 'n/a' ---handling missing data
    END as cst_gndr, ---normalize gender values to readable format
    cst_create_date
  from
    (
      select
        *,
        row_number() over (partition by cst_id order by cst_create_date desc) as flag_last
      from
        bronze.crm_cust_info
    )
  where
    flag_last = 1
    and cst_id is not null --select the most recent record per customer(removge duplicate) and remove null

In [0]:
%sql
select * from silver.crm_cust_info


### crm_prd_info

In [0]:
%sql
select * from bronze.crm_prd_info

In [0]:
%sql
--prd_id is primary key
--check for null and duplicates in primary key
select prd_id,count(*) from bronze.crm_prd_info group by prd_id having count(*)> 1 or prd_id is null

In [0]:

%sql
--check for unwanted space
select prd_id,
prd_key,
replace(substring(prd_key,1,5),'-','_') as cat_id,
replace(substring(prd_key,7,length(prd_key)),'-','_') as prd_key,
prd_nm,
prd_cost,
prd_line,
prd_start_dt,
prd_end_dt   from bronze.crm_prd_info
where prd_nm != trim(prd_nm)

In [0]:
%sql
---check for nulls or negative number
---Expected :No result
select prd_id,
prd_key,
replace(substring(prd_key,1,5),'-','_') as cat_id,
replace(substring(prd_key,7,length(prd_key)),'-','_') as prd_key,
prd_nm,
prd_cost,
prd_line,
prd_start_dt,
prd_end_dt   from bronze.crm_prd_info
where prd_cost <= 0 or prd_cost is null
---Since we have null value as per business we need to make it 0
    

In [0]:
---Need to check for abbrevation change or not like we did for prd_lines
--Data standardization & consistency
select prd_id,
prd_key,
replace(substring(prd_key,1,5),'-','_') as cat_id,
replace(substring(prd_key,7,length(prd_key)),'-','_') as prd_key,
prd_nm,
coalesce(prd_cost,0) as prd_cost,
case upper(trim(prd_line)) 
when 'M' then 'Mountain'
when 'R' then 'Road'
when 'S' then 'other Sales'
when 'T' then 'Touring'
else 'n/a' end as prd_line,
prd_start_dt,
prd_end_dt   from bronze.crm_prd_info
where prd_cost <= 0 or prd_cost is null


In [0]:
-- check for invalid date orders
---start date < end date
---end date should be small to start date of next recird against same id
select prd_id,
prd_key,
prd_nm,
prd_start_dt,
prd_end_dt ,
lead(prd_start_dt) over (partition by prd_key order by prd_start_dt)-1 as prd_end_dt_test
  from bronze.crm_prd_info
where  prd_key in ('AC-HE-HL-U509-R' , 'AC-HE-HL-U509')


In [0]:
-- check for invalid date orders
select prd_id,
prd_key,
replace(substring(prd_key,1,5),'-','_') as cat_id,
replace(substring(prd_key,7,length(prd_key)),'-','_') as prd_key,
prd_nm,
coalesce(prd_cost,0) as prd_cost,
case upper(trim(prd_line)) 
when 'M' then 'Mountain'
when 'R' then 'Road'
when 'S' then 'other Sales'
when 'T' then 'Touring'
else 'n/a' end as prd_line,
prd_start_dt,
lead(prd_start_dt) over (partition by prd_key order by prd_start_dt)-1 as prd_end_dt   
from bronze.crm_prd_info



### > - crm_prd_info

In [0]:
--Modify table defination
CREATE OR REPLACE TABLE silver.crm_prd_info (
  prd_id INT,
  cat_id VARCHAR(50),
  prd_key VARCHAR(50),
  prd_nm VARCHAR(50),
  prd_cost INT,
  prd_line VARCHAR(50),
  prd_start_dt TIMESTAMP,
  prd_end_dt TIMESTAMP,
  dwh_creat_date TIMESTAMP DEFAULT current_timestamp())
USING delta
TBLPROPERTIES (
  'delta.columnMapping.mode' = 'name',
  'delta.enableDeletionVectors' = 'true',
  'delta.feature.allowColumnDefaults' = 'supported',
  'delta.feature.appendOnly' = 'supported',
  'delta.feature.changeDataFeed' = 'supported',
  'delta.feature.checkConstraints' = 'supported',
  'delta.feature.columnMapping' = 'supported',
  'delta.feature.deletionVectors' = 'supported',
  'delta.feature.generatedColumns' = 'supported',
  'delta.feature.invariants' = 'supported',
  'delta.minReaderVersion' = '3',
  'delta.minWriterVersion' = '7')



## Data insert into silver table

- > Data Transformation 
- Deriverd columns 
> - Extract category id 
> - Extract product key
- Handling missing values
> - is null to 0
- Data normalization
> - Map product lines code to descriptive values
- Data type casting
> - convert to date
> - Data enrichment > add new data for data anlsysis like prd_end_date using lead()

In [0]:
insert overwrite silver.crm_prd_info(prd_id,cat_id,prd_key,prd_nm,prd_cost,prd_line,prd_start_dt,prd_end_dt) 
select prd_id,
replace(substring(prd_key,1,5),'-','_') as cat_id,
replace(substring(prd_key,7,length(prd_key)),'_','-') as prd_key,
prd_nm,
coalesce(prd_cost,0) as prd_cost,
case upper(trim(prd_line)) 
when 'M' then 'Mountain'
when 'R' then 'Road'
when 'S' then 'other Sales'
when 'T' then 'Touring'
else 'n/a' end as prd_line,
prd_start_dt,
lead(prd_start_dt) over (partition by prd_key order by prd_start_dt)-1 as prd_end_dt   
from bronze.crm_prd_info



In [0]:
SELECT * FROM SILVER.crm_prd_info

## Quality Check of silver Table

##### Check for primary key



In [0]:
%sql
select
  prd_id,
  count(*)
from
  silver.crm_prd_info
group by
  prd_id
having
  count(*) > 1
  or prd_id is null

##### check for unwanted spaces

In [0]:
%sql
select
  prd_nm
from
  silver.crm_prd_info
where prd_nm != TRIM(prd_nm)

##### Check for Nulls or Negative Numbers

In [0]:
select prd_cost from   silver.crm_prd_info
where  prd_cost < 0 OR prd_cost is null

##### Data standardization & consistency

In [0]:
select distinct prd_line from silver.crm_prd_info


##### Check for invalid data orders



In [0]:
select * from silver.crm_prd_info where prd_end_dt < prd_start_dt

### > - crm_sales_details

In [0]:
%sql
select sls_ord_num,
sls_prd_key,
sls_cust_id,
sls_order_dt,
sls_ship_dt,
sls_due_dt,
sls_sales,
sls_quantity,
sls_price 
from bronze.crm_sales_details
where sls_ord_num !=  TRIM(sls_ord_num)

In [0]:
%sql
select sls_ord_num,
sls_prd_key,
sls_cust_id,
sls_order_dt,
sls_ship_dt,
sls_due_dt,
sls_sales,
sls_quantity,
sls_price 
from bronze.crm_sales_details
where sls_prd_key  NOT IN (SELECT prd_key FROM SILVER.crm_prd_info)

In [0]:
%sql
select sls_ord_num,
sls_prd_key,
sls_cust_id,
sls_order_dt,
sls_ship_dt,
sls_due_dt,
sls_sales,
sls_quantity,
sls_price 
from bronze.crm_sales_details
where sls_cust_id  NOT IN (SELECT cst_id FROM SILVER.crm_cust_info)

In [0]:
--check for invalid dates
select
  sls_ord_num,
  sls_prd_key,
  sls_cust_id,
  CASE
    WHEN
      (
        sls_order_dt = 0
        OR LEN(sls_order_dt) != 8
      )
    THEN
      NULL
    ELSE to_date(CAST(sls_order_dt AS STRING), 'yyyyMMdd')
  END as sls_order_dt,
  CASE
    WHEN
      (
        sls_ship_dt = 0
        OR LEN(sls_ship_dt) != 8
      )
    THEN
      NULL
    ELSE to_date(CAST(sls_ship_dt AS STRING), 'yyyyMMdd')
  END sls_ship_dt,
    CASE
    WHEN
      (
        sls_due_dt = 0
        OR LEN(sls_due_dt) != 8
      )
    THEN
      NULL
    ELSE to_date(CAST(sls_due_dt AS STRING), 'yyyyMMdd')
  END sls_due_dt,
  sls_sales,
  sls_quantity,
  sls_price
from
  bronze.crm_sales_details

In [0]:
--check for business rules sales  =  quantity * price ,negative zeros ,null are not aloud
select
  sls_ord_num,
  sls_prd_key,
  sls_cust_id,
  CASE
    WHEN
      (
        sls_order_dt = 0
        OR LEN(sls_order_dt) != 8
      )
    THEN
      NULL
    ELSE to_date(CAST(sls_order_dt AS STRING), 'yyyyMMdd')
  END as sls_order_dt,
  CASE
    WHEN
      (
        sls_ship_dt = 0
        OR LEN(sls_ship_dt) != 8
      )
    THEN
      NULL
    ELSE to_date(CAST(sls_ship_dt AS STRING), 'yyyyMMdd')
  END sls_ship_dt,
    CASE
    WHEN
      (
        sls_due_dt = 0
        OR LEN(sls_due_dt) != 8
      )
    THEN
      NULL
    ELSE to_date(CAST(sls_due_dt AS STRING), 'yyyyMMdd')
  END sls_due_dt,
  sls_sales,
  sls_quantity,
  sls_price
from
  bronze.crm_sales_details
  where sls_sales != sls_quantity * sls_price or sls_sales is null or sls_quantity is null or sls_price is null
  or sls_sales <= 0 or sls_quantity <= 0 or sls_price <= 0

###
Based on decision 
>- If sales is negative ,zero,or null ,derive it using quantity and price
> - if price is zero or null ,calculate it using sales and quantity
> - if price is negative ,convert it into positive values

In [0]:
--check for business rules sales  =  quantity * price ,negative zeros ,null are not aloud
select
  sls_ord_num,
  sls_prd_key,
  sls_cust_id,
  CASE
    WHEN
      (
        sls_order_dt = 0
        OR LEN(sls_order_dt) != 8
      )
    THEN
      NULL
    ELSE to_date(CAST(sls_order_dt AS STRING), 'yyyyMMdd')
  END as sls_order_dt,
  CASE
    WHEN
      (
        sls_ship_dt = 0
        OR LEN(sls_ship_dt) != 8
      )
    THEN
      NULL
    ELSE to_date(CAST(sls_ship_dt AS STRING), 'yyyyMMdd')
  END sls_ship_dt,
    CASE
    WHEN
      (
        sls_due_dt = 0
        OR LEN(sls_due_dt) != 8
      )
    THEN
      NULL
    ELSE to_date(CAST(sls_due_dt AS STRING), 'yyyyMMdd')
  END sls_due_dt,
  CASE WHEN  sls_sales <= 0 OR sls_sales is null or sls_sales != sls_quantity * abs(sls_price) then  sls_quantity * abs(sls_price)  else sls_sales end as sls_sales,
  sls_quantity,
  CASE WHEN  sls_price <= 0 OR sls_price is null or sls_price != abs(sls_sales) / sls_quantity   then abs(sls_sales) / sls_quantity else sls_price end as sls_price
  
from
  bronze.crm_sales_details
  -- where sls_sales != sls_quantity * sls_price or sls_sales is null or sls_quantity is null or sls_price is null
  -- or sls_sales <= 0 or sls_quantity <= 0 or sls_price <= 0

In [0]:
CREATE OR REPLACE TABLE workspace.silver.crm_sales_details (
  sls_ord_num VARCHAR(50),
  sls_prd_key VARCHAR(50),
  sls_cust_id INT,
  sls_order_dt DATE,
  sls_ship_dt DATE,
  sls_due_dt DATE,
  sls_sales INT,
  sls_quantity INT,
  sls_price INT,
  dwh_creat_date TIMESTAMP DEFAULT current_timestamp())
USING delta
TBLPROPERTIES (
  'delta.columnMapping.mode' = 'name',
  'delta.enableDeletionVectors' = 'true',
  'delta.feature.allowColumnDefaults' = 'supported',
  'delta.feature.appendOnly' = 'supported',
  'delta.feature.changeDataFeed' = 'supported',
  'delta.feature.checkConstraints' = 'supported',
  'delta.feature.columnMapping' = 'supported',
  'delta.feature.deletionVectors' = 'supported',
  'delta.feature.generatedColumns' = 'supported',
  'delta.feature.invariants' = 'supported',
  'delta.minReaderVersion' = '3',
  'delta.minWriterVersion' = '7')


In [0]:
INSERT OVERWRITE workspace.silver.crm_sales_details
(sls_ord_num,
sls_prd_key,
sls_cust_id,
sls_order_dt,
sls_ship_dt,
sls_due_dt,
sls_sales,
sls_quantity,
sls_price)
--check for business rules sales  =  quantity * price ,negative zeros ,null are not aloud
select
  sls_ord_num,
  sls_prd_key,
  sls_cust_id,
  CASE
    WHEN
      (
        sls_order_dt = 0
        OR LEN(sls_order_dt) != 8
      )
    THEN
      NULL
    ELSE to_date(CAST(sls_order_dt AS STRING), 'yyyyMMdd')
  END as sls_order_dt,
  CASE
    WHEN
      (
        sls_ship_dt = 0
        OR LEN(sls_ship_dt) != 8
      )
    THEN
      NULL
    ELSE to_date(CAST(sls_ship_dt AS STRING), 'yyyyMMdd')
  END sls_ship_dt,
    CASE
    WHEN
      (
        sls_due_dt = 0
        OR LEN(sls_due_dt) != 8
      )
    THEN
      NULL
    ELSE to_date(CAST(sls_due_dt AS STRING), 'yyyyMMdd')
  END sls_due_dt,
  CASE WHEN  sls_sales <= 0 OR sls_sales is null or sls_sales != sls_quantity * abs(sls_price) then  sls_quantity * abs(sls_price)  else sls_sales end as sls_sales,
  sls_quantity,
  CASE WHEN  sls_price <= 0 OR sls_price is null   then abs(sls_sales) / sls_quantity else sls_price end as sls_price
  
from
  bronze.crm_sales_details


In [0]:
%sql
select * from workspace.silver.crm_sales_details
  where sls_sales != sls_quantity * sls_price or sls_sales is null or sls_quantity is null or sls_price is null
  or sls_sales <= 0 or sls_quantity <= 0 or sls_price <= 0

Building ERP tables

### erp_cust_az12

In [0]:
%sql
select cid,bdate,gen from bronze.erp_cust_az12

In [0]:
%sql
select
  TRIM(replace(cid, 'NAS', '')) as cid_suffix,
  bdate,
  gen
from
  bronze.erp_cust_az12 


In [0]:
%sql
---DATE OUT OF RANGE
select
  TRIM(replace(cid, 'NAS', '')) as cid_suffix,
  CASE WHEN  bdate >  GETDATE() THEN NULL ELSE  bdate END AS bdate,
  gen
from
  bronze.erp_cust_az12 


In [0]:


%sql
---DATA STANDARDIZATION & CONSISTENCY
insert overwrite   silver.erp_cust_az12(cid,bdate,gen) 
select
  TRIM(replace(cid, 'NAS', '')) as cid, ---Remove NAS prefix
  CASE WHEN  bdate >  GETDATE() THEN NULL ELSE  bdate END AS bdate, --set future date to NULL
 case when trim(upper(gen)) = 'M' then 'Male' when trim(upper(gen)) = 'F' then 'Female' else 'n/a' end as gen --Set normalize gender values and handle unknown cases
from
  bronze.erp_cust_az12 



In [0]:
select * from silver.erp_cust_az12

### erp_loc_a101

In [0]:
select cid,cntry from bronze.erp_loc_a101 where cid not in (select cid from silver.erp_cust_az12)

In [0]:
insert overwrite  silver.erp_loc_a101(cid,cntry)
select
  replace(cid, '-', '') as cid,---Remove -
  case
    when trim(cntry) = 'DE' then 'Germany'
    when trim(cntry) in ('US', 'USA') then 'United States'
    when trim(cntry) = ''
    or trim(cntry) is null then 'n/a'
    else trim(cntry)
  end as cntry ---standardization & consistency ---Normalize and handling missing or blank country codes
from
  bronze.erp_loc_a101

In [0]:
select
distinct cntry

from
silver.erp_loc_a101

###
erp_px_cat_g1v2

In [0]:
%sql
insert overwrite    silver.erp_px_cat_g1v2 (id,
  cat,
  subcat,
  MAINTENANCE)
select
  id,
  cat,
  subcat,
  MAINTENANCE
from
  bronze.erp_px_cat_g1v2

In [0]:
%sql
select
 *
from
    silver.erp_px_cat_g1v2 